In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader = WebBaseLoader('https://docs.smith.langchain.com')
docs = loader.load()
document = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [5]:
vectordb = Chroma.from_documents(document, HuggingFaceEmbeddings())

/Users/amanatanayak/Documents/Python3/LLM/LANGCHAIN_TUTORIAL/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/amanatanayak/Documents/Python3/LLM/LANGCHAIN_TUTORIAL/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
retriver = vectordb.as_retriever()
retriver

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x166359550>, search_kwargs={})

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, 'langsmith_search','Search for information about LangSmith. For any questions about LangSmith, you must use this tool!')


In [8]:
retriver_tool.name

'langsmith_search'

In [9]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [10]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [11]:
tools = [wiki, arxiv, retriver_tool]

### Agents 
Is to use a language model to choose a sequence of action to take. In chains, a sequence of actions is hardcoded(in code). In agents, a language model is used as a reasoning engine to determin which actions to take in which order

In [12]:
from langchain.agents import create_openapi_agent
from lang